In [ ]:
# Paso 0: Cargar el archivo de Excel
from google.colab import files
uploaded = files.upload()  # Subir el archivo .xlsx con los datos


In [ ]:
# Paso 1: Importar librerías necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
# Paso 2: Cargar el archivo de Excel
df = pd.read_excel("venta_iphone_data.xlsx")  # Cambia el nombre si tu archivo tiene otro
df.head(5)  # Mostrar primeras filas del dataset


In [ ]:
# Paso 3: Análisis exploratorio de los datos
print(df.info())           # Ver tipos de datos y valores nulos
print(df.describe())       # Estadísticas descriptivas
print(df.isnull().sum())   # Revisión de nulos por columna


In [ ]:
# Visualización de la distribución de precios en COP
sns.histplot(df["Precio (COP)"], kde=True, bins=30)
plt.title("Distribución del Precio (COP)")
plt.xlabel("Precio (COP)")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()


In [ ]:
# Distribución de otras variables numéricas
num_cols = ["Modelo","Capacidad (GB)", "Años de uso", "Batería (%)", "RAM (GB)", "Cámaras (MP)"]

for col in num_cols:
    plt.figure(figsize=(6, 4))
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.grid(True)
    plt.show()


In [ ]:
# Boxplot del precio por modelo
plt.figure(figsize=(12, 6))
sns.boxplot(x="Modelo", y="Precio (COP)", data=df)
plt.title("Precio (COP) por Modelo")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [ ]:
# Boxplot del precio por estado (nuevo/usado)
sns.boxplot(x="Estado", y="Precio (COP)", data=df)
plt.title("Precio (COP) por Estado")
plt.grid(True)
plt.show()


In [ ]:
# Boxplot del precio por estado (nuevo/usado)
sns.boxplot(x="Estado general", y="Precio (COP)", data=df)
plt.title("Precio (COP) por Estado general")
plt.grid(True)
plt.show()

In [ ]:
# Paso 4: Preprocesamiento de variables categóricas
from sklearn.preprocessing import OrdinalEncoder

def codificar_datos_dispositivo(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica codificación ordinal a 'Estado general' y codificación one-hot a otras
    columnas categóricas especificadas en un DataFrame.

    Args:
        df (pd.DataFrame): El DataFrame de entrada que contiene los datos del dispositivo.

    Returns:
        pd.DataFrame: El DataFrame con las columnas especificadas codificadas.
    """
    # Definir el orden lógico para los valores de 'Estado general'
    orden_estado_general = [["Bueno", "Medio", "Malo"]]

    # Inicializar y aplicar el codificador ordinal
    ordinal_encoder = OrdinalEncoder(categories=orden_estado_general)
    df[["Estado general"]] = ordinal_encoder.fit_transform(df[["Estado general"]])

    # Convertir variables categóricas a numéricas usando one-hot encoding
    df_encoded = pd.get_dummies(df, columns=[
        "Estado", "Piezas originales", "Tiene caja", "Face ID"
    ], drop_first=True)

    return df_encoded

#Llamar funcion de transformación
df_encoded = codificar_datos_dispositivo(df)

# Separar características (X) y variable objetivo (y)
X = df_encoded.drop(["Precio (COP)"], axis=1)
y = df_encoded["Precio (COP)"]


In [ ]:
#Imprimir datos X
X.head()

In [ ]:
#Exportar datos normalizados y transformados
from google.colab import sheets
#Genera nuevo archivo con datos normalizados
#sheet = sheets.InteractiveSheet(df=X)

In [ ]:
#Matriz de correlacion X
corr_matrix = X.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de Correlación")
plt.show()

In [ ]:
# Paso 5: Dividir los datos en entrenamiento (70%), validación (15%) y prueba (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#Mostrar cuantos datos
print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de validación: {len(X_val)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")

In [ ]:
# Paso 6: Entrenamiento del modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)  # Ajustar el modelo con los datos de entrenamiento


In [ ]:
# Paso 7: Evaluación del modelo en datos de validación
y_val_pred = model.predict(X_val)

# Calcular y mostrar métricas
# Error cuadratico medio
print("MSE:", mean_squared_error(y_val, y_val_pred))
# Presicion
#El R² es una métrica fundamental en regresión que te indica qué tan bien tu modelo se ajusta a los datos
print("R2:", r2_score(y_val, y_val_pred))
#Raíz del Error Cuadrático Medio
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_val_pred)))

R2: Cuanto más cerca esté de 1, mejor se ajusta el modelo.

RMSE: Esto significa que, en promedio, tus predicciones de precios de iPhones
están desviadas por aproximadamente $36,561 COP del precio real.

In [ ]:
# Paso 8: Evaluación final en el conjunto de prueba
y_test_pred = model.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_test_pred))
print("R2:", r2_score(y_test, y_test_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))

In [ ]:
# Paso 9: Visualización de las predicciones vs valores reales
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("Precio Real (COP)")
plt.ylabel("Precio Predicho (COP)")
plt.title("Comparación: Precio Real vs Precio Predicho")
plt.grid(True)
plt.show()


Ingrese las características del iPhone:

1. Capacidad (GB): 128
2. Años de uso: 1
3. Porcentaje de batería (%): 85
4. RAM (GB): 4
5. Cámaras (MP): 12
6. Modelo (ej: 11, 12, etc): 12
7. Estado (Nuevo/Usado): Usado
8. ¿Tiene caja? (Sí/No): Sí
9. ¿Tiene Face ID? (Sí/No): Sí







In [ ]:
# ✅ Ejemplo: Predicción del precio de un iPhone con entrada fija

nuevo_raw = {
    "Capacidad (GB)": 256,
    "Años de uso": 1,
    "Batería (%)": 85,
    "RAM (GB)": 6,
    "Cámaras (MP)": 48,
    "Modelo": 15,
    "Estado": "Usado",
    "Estado general": "Bueno",
    "Tiene caja": "Sí",
    "Piezas originales": "Sí",
    "Face ID": "Sí"  # si usaste esta columna también
}

nuevo_df = pd.DataFrame([nuevo_raw])

# Llamar función de transformacion
nuevo_encoded = codificar_datos_dispositivo(nuevo_df)

# Rellenar columnas faltantes con 0
for col in X.columns:
    if col not in nuevo_encoded.columns:
        nuevo_encoded[col] = 0

# Reordenar las columnas igual que en el entrenamiento
nuevo_encoded = nuevo_encoded[X.columns]

precio_estimado = model.predict(nuevo_encoded)[0]
print(f"📱 Precio estimado del iPhone: ${int(precio_estimado):,} COP")
